In [ ]:
import pandas as pd
import ipywidgets as widgets
import datetime as dt
import seaborn as sns
from datetime import datetime
from IPython.display import display, clear_output
from IPython.core.display import HTML
import random
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap, HeatMapWithTime
import tempfile
import numpy as np
from functools import partial
import re
from sklearn.cluster import KMeans
import plotly.express as px
import plotly
import plotly.graph_objects as go
import plotly.offline as pyo
from IPython.display import display as ipDisplay


In [ ]:
t = go.FigureWidget()
type(t)

In [ ]:
test = px.area(x=[1,2,3], y=[6,5,4])
type(test)

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
class app:
    def __init__(self, main) -> None:
        self.main = main
        
    def _build_layout(self):
        self.output_main = widgets.Output()
        self.layout = widgets.VBox(children=[self.output_main])
        return self.layout
    
    def _filtrar_pedidos(self, status = None, statusIs = '=='):
        self._atualizar_filtros()
        pedidos = self.main.pedidos.query(
            f"@self.dataInicial <= ano_mes_cobranca <= @self.dataFinal and id_praca in @self.pracas")
        if status != None:
            pedidos = pedidos.query(f"status {statusIs} @status")
        return pedidos
    
    def _filtrar_artigosNoPedido(self, status = "FECHADO"):
        self._atualizar_filtros()
        artigosNoPedido =     self.main.artigosPedido.query(
            f"ref in @self.artigosFiltrar and @self.dataInicial <= ano_mes_cobranca <= @self.dataFinal and id_praca in @self.pracas and status == @status"
        )
        return artigosNoPedido
        
    def _filtrar_desempenho(self):
        self._atualizar_filtros()
        desempenho = self.main.desempenhoPraca.query("@self.dataInicial <= ano_mes_cobranca <= @self.dataFinal and id_vendedor in @self.promotores and id_praca in @self.pracas")
        return desempenho
    
    def _get_listaVendedorasAtivas(self):
        self._atualizar_filtros()
        pedidos = self.main.pedidos.query('id_praca in @self.pracas and status == "DIGITADO"')
        lista_vendedorasAtivas = pedidos['id_sacoleira'].tolist()
        return lista_vendedorasAtivas
    
    def _get_itensVendedorasAtivas(self):
        listaAtiva = self._get_listaVendedorasAtivas()
        histArtigosAtivas = self.main.artigosPedido.query("id_vendedora in @listaAtiva")
        return histArtigosAtivas
    
    def _atualizar_exibicao(self, exibicoes, output=None):
        """
        Args:
            exibicoes (list): Uma lista contendo um ou mais dicionario de exibicao,
            exibicao (dic): Objeto a ser exibido em uma tab sendo seus argumentos:
            * 'nome' : Nome da tab;
            * 'objeto' : Objeto a ser exibido;
            * 'style' : parametros de estilo para dataframes;
            output (_type_, optional): _description_. Defaults to None.
        """
        if output is not None:
            output = output
        else:
            output = self.output_main
        if not isinstance(exibicoes, list):
            exibicoes = [exibicoes]
            
        with output:
            clear_output()
            tab_contents = [widgets.Output() for _ in range(len(exibicoes))]
            children = [widgets.VBox([tab]) for tab in tab_contents]
            tab = widgets.Tab()
            tab.children = children
            for i, extra in enumerate(exibicoes):
                tab.set_title(i, exibicoes[i]["nome"])
            display(tab)

            for i, exibicao in enumerate(exibicoes):
                with tab_contents[i]:
                    if isinstance(exibicao['objeto'], list):
                        for item in exibicao['objeto']:
                            display(item)
                    elif isinstance(exibicao['objeto'], folium.Map):
                        display(HTML(f'<div style="width: 1600px; height: 700px;">{exibicao["objeto"]._repr_html_()}</div>'))
                        
                    elif isinstance(exibicao['objeto'], plotly.graph_objs._figurewidget.FigureWidget):
                        clear_output()
                        button = widgets.Button(description="Exibir Treemap")
                        fig=exibicao['objeto']
                        def show_fig(_):
                            with output:
                                clear_output()
                                display(fig)
                        button.on_click(show_fig)
                        display(button)
                    elif isinstance(exibicao['objeto'], pd.DataFrame):
                        def criar_funcao_atualizar(exibicao, dropdown_coluna, dropdown_ascDesc, output):
                            def atualizar_dataframe(change):
                                dataframe = exibicao['objeto']
                                colunas = [coluna for coluna in exibicao['objeto'].columns]
                                if dropdown_coluna.value in colunas:
                                    dataframe_final = dataframe.sort_values(by=dropdown_coluna.value, ascending=dropdown_ascDesc.value)
                                else:
                                    dataframe_final = dataframe.sort_index(level=dropdown_coluna.value, ascending=dropdown_ascDesc.value)
                                if "style" in exibicao:
                                    dataframe_final = dataframe_final.style.format(exibicao['style'])
                                with output:
                                    clear_output()
                                    display(dataframe_final)
                            return atualizar_dataframe
                        def criar_exportar_exibicao(objeto):
                            def exportar_exibicao(objeto):
                                agora = datetime.now()
                                export = agora.strftime('%Y-%m-%d_%H-%M-%S')
                                dataframe = exibicao['objeto']
                                dataframe.to_excel(f"export/dataframe_exportado_{export}.xlsx", index=True)
                                
                        exibicao = exibicao
                        colunas_exb = []
                        for index in exibicao['objeto'].index.names:
                            colunas_exb.append(index)
                        for coluna in exibicao['objeto'].columns:
                            colunas_exb.append(coluna)
                        dropdown_coluna_exibicao = widgets.Dropdown(description='Campo', options=colunas_exb)
                        dropdown_ascDesc_exibicao = widgets.Dropdown(description='Orientação', options=[('Ascendente', True), ('Descendente', False)])
                        button_ordenar_exibicao = widgets.Button(description='Ordenar')
                        box_ordem_exibicao = widgets.HBox(children=[dropdown_coluna_exibicao, dropdown_ascDesc_exibicao, button_ordenar_exibicao])
                        button_exportar_exibicao = widgets.Button(description="Exportar")
                        output_exibicao = widgets.Output()
                        box_main = widgets.VBox(children=[box_ordem_exibicao, button_exportar_exibicao, output_exibicao])
                        funcao_atualizar_dataframe = criar_funcao_atualizar(exibicao, dropdown_coluna_exibicao, dropdown_ascDesc_exibicao, output_exibicao)
                        funcao_exportar = criar_exportar_exibicao(exibicao['objeto'])
                        button_ordenar_exibicao.on_click(funcao_atualizar_dataframe)
                        button_exportar_exibicao.on_click(funcao_exportar)
                        display(box_main)
                        funcao_atualizar_dataframe(None)  # O segundo argumento (change) será preenchido automaticamente pelo widget
                    else:
                        display(exibicao['objeto'])
    
    def _atualizar_filtros(self):
        self.artigosFiltrar = self.main.filtros['artigos']
        self.dataInicial = self.main.filtros['data inicial']
        self.dataFinal = self.main.filtros['data final']
        self.promotores = self.main.filtros['promotores']
        self.pracas = self.main.filtros['pracas']
        
    def _retornar_funcoes(self):
        def normalize(funcao_chamavel):
            normalize = funcao_chamavel.replace("_", " ")
            normalize = normalize.title()
            return normalize
        atributos = dir(self)
        todas_funcoes = [atributo for atributo in atributos if callable(getattr(self, atributo))]
        funcoes_externas = [funcao for funcao in todas_funcoes if funcao[0] != '_']
        funcoes_chamaveis = []
        for funcao in funcoes_externas:
            funcoes_chamaveis.append((normalize(funcao), funcao))
        return funcoes_chamaveis
        

In [ ]:
class relatorio(app):
    """
    21 - 01/09/2020 - 01/08/2021 ~
    22 - 01/09/2021 - 01/08/2022 ~
    23 - 01/09/2022 - 01/08/2023
    """
    def __init__(self, main) -> None:
        super().__init__(main)
    
    def venda_periodo(self):
        pedidos = self._filtrar_pedidos().groupby('status')
        df = pd.DataFrame()
        df['Total Consignado'] = pedidos['valor_total_pedido'].sum()
        df['Total Pedidos'] = pedidos['id_pedido'].count()
        df['Total Vendido'] = pedidos['valor_venda_pedido'].sum()
        
        style = {
            'Total Consignado': 'R$ {:,.2f}',
            'Total Pedidos': '{:.0f}',
            'Total Vendido': 'R$ {:,.2f}'
        }
        self._atualizar_exibicao({'nome':'Relatorio', 'objeto':df, 'style': style})
    
    def total_na_rua(self):
        pedidos = self._filtrar_pedidos().groupby('status')
        df = pd.DataFrame()
        df['Total Consignado'] = pedidos['valor_total_pedido'].sum()
        df['Total Pedidos'] = pedidos['id_pedido'].count()
        
        style = {
            'Total Consignado': 'R$ {:,.2f}',
            'Total Pedidos': '{:.0f}'
        }
        self._atualizar_exibicao({'nome':'Relatorio', 'objeto':df, 'style': style})
        
        
    def top_10_artigos_venda(self):
        artigos = self._filtrar_artigosNoPedido().groupby(['ref', 'artigo']).agg({'quantidade_entregue': 'sum', 'quantidade_vendida': 'sum', 'valor_vendido': 'sum'})
        top10 = artigos.nlargest(10, 'quantidade_vendida')
        style = {
            'valor_vendido': 'R$ {:,.2f}',
            'quantidade_entregue': '{:.0f}',
            'quantidade_vendida': '{:.0f}',
            'Rep Faturamento' : '{:.2%}'
        }
        self._atualizar_exibicao({'nome':'Relatorio', 'objeto':top10})

    def top_10_artigos_faturamento(self):
        artigos = self._filtrar_artigosNoPedido().groupby(['ref', 'artigo']).agg({'quantidade_entregue': 'sum', 'quantidade_vendida': 'sum', 'valor_vendido': 'sum'})
        top10 = artigos.nlargest(10, 'valor_vendido')
        top10['Rep Faturamento'] = top10['valor_vendido'] / artigos['valor_vendido'].sum()
        style = {
            'valor_vendido': 'R$ {:,.2f}',
            'quantidade_entregue': '{:.0f}',
            'quantidade_vendida': '{:.0f}',
            'Rep Faturamento' : '{:.2%}'
        }
        self._atualizar_exibicao({'nome':'Relatorio', 'objeto':top10, 'style': style})
        
    def dados_itens_fora_top_10(self):
        artigos = self._filtrar_artigosNoPedido().groupby(['ref', 'artigo']).agg({'quantidade_entregue': 'sum', 'quantidade_vendida': 'sum', 'valor_vendido': 'sum'})
        artigos.reset_index(inplace=True)  # Certifique-se de atribuir de volta a "artigos" após reset_index
        top10 = artigos.nlargest(10, 'valor_vendido')
        listaTop10 = top10['ref'].tolist()
        
        # Filtrar artigos que não estão no top 10
        artigos_fora_top10 = artigos[~artigos['ref'].isin(listaTop10)]
        artigos_fora_top10 = pd.merge(artigos_fora_top10, self.main.artigos[['ref', 'descricao_grupo']], on='ref')
        artigos_fora_top10g = artigos_fora_top10.groupby('descricao_grupo')
        
        # Grupos fora 
        df = pd.DataFrame()
        df['Quantidade de Itens'] = artigos_fora_top10g['ref'].count()
        df['Total Vendido R$'] = artigos_fora_top10g['valor_vendido'].sum()
        df['Total Un Vendidos'] = artigos_fora_top10g['quantidade_vendida'].sum()
        df['Rep Total Vendido'] = df['Total Vendido R$'] / artigos['valor_vendido'].sum()
        df['Total Consignado'] = artigos_fora_top10g['quantidade_entregue'].sum()
        df['% Vendida'] = df['Total Un Vendidos']/df['Total Consignado']
        
        style = {
            'Total Vendido R$': 'R$ {:,.2f}',
            'Total Un Vendidos': '{:.0f}',
            'Quantidade de Itens': '{:.0f}',
        }
        
        self._atualizar_exibicao({'nome': 'Relatorio', 'objeto': df, 'style': style})
    
    def analise_grupos(self):
        artigos = self._filtrar_artigosNoPedido().groupby(['ref', 'artigo']).agg({'quantidade_entregue': 'sum', 'quantidade_vendida': 'sum', 'valor_vendido': 'sum'})
        artigos.reset_index(inplace=True)  # Certifique-se de atribuir de volta a "artigos" após reset_index
        artigos = pd.merge(artigos, self.main.artigos[['ref', 'descricao_grupo']], on='ref')
        grupos = artigos.groupby('descricao_grupo')
        
        df = pd.DataFrame()
        df['Quantidade de Itens'] = grupos['ref'].count()
        df['Total Vendido R$'] = grupos['valor_vendido'].sum()
        df['Total Un Vendidos'] = grupos['quantidade_vendida'].sum()
        df['Rep Total Vendido'] = df['Total Vendido R$'] / artigos['valor_vendido'].sum()
        df['Total Consignado'] = artigos['quantidade_entregue'].sum()
        df['% Vendida'] = df['Total Un Vendidos']/df['Total Consignado']
        
        self._atualizar_exibicao({'nome': 'Relatorio', 'objeto': df})
        
    def desempenho_periodo_promotor(self):
        desempenhos = self._filtrar_desempenho()
        gp_desempenhos = desempenhos.groupby('descricao_vendedor')
        df = pd.DataFrame()
        df['Praças Cobradas'] = gp_desempenhos['id_desempenhopraca'].count()
        df['Total Recebido'] = gp_desempenhos['valor_recebido'].sum()
        df['Total Pepinos'] = gp_desempenhos['total_novos_pepinos'].sum()
        df['Vendedoras Cobradas'] = gp_desempenhos['pedidos_fechados'].sum()
        df['Recebimento x Acertos'] = df['Total Recebido'] / df['Praças Cobradas']
        df['Total Consignado'] = gp_desempenhos['vl_total_pedidos_liquido'].sum()
        self._atualizar_exibicao({'nome': 'Relatorio', 'objeto': df})
        
    def exprtavel_cidades_por_praca(self):
        agora = datetime.now()
        export = agora.strftime('%Y-%m-%d_%H-%M')
        pedidos = self._filtrar_pedidos()
        with pd.ExcelWriter(f"export/cidades-pracas_{export}.xlsx", engine='xlsxwriter') as writer:
            for promotor in pedidos['descricao_vendedor'].unique():
                pedidosPromotor = pedidos[pedidos['descricao_vendedor']==promotor]
                pedidosPromotorGP = pedidosPromotor.groupby(['descricao_praca', 'cidade'])
                df = pd.DataFrame()
                df['Pedidos'] = pedidosPromotorGP['id_pedido'].count()
                df['Total Deixado'] = pedidosPromotorGP['valor_total_pedido'].sum()
                df['Total Vendido'] = pedidosPromotorGP['valor_venda_pedido'].sum()
                df.to_excel(writer, sheet_name=f'{promotor[:-8]}', index=True)
        
    def exprtavel_cidades(self):
        agora = datetime.now()
        export = agora.strftime('%Y-%m-%d_%H-%M')
        pedidos = self._filtrar_pedidos()
        with pd.ExcelWriter(f"export/cidades_{export}.xlsx", engine='xlsxwriter') as writer:
            for promotor in pedidos['descricao_vendedor'].unique():
                pedidosPromotor = pedidos[pedidos['descricao_vendedor']==promotor]
                pedidosPromotorGP = pedidosPromotor.groupby(['cidade'])
                df = pd.DataFrame()
                df['Pedidos'] = pedidosPromotorGP['id_pedido'].count()
                df['Total Deixado'] = pedidosPromotorGP['valor_total_pedido'].sum()
                df['Total Vendido'] = pedidosPromotorGP['valor_venda_pedido'].sum()
                df.to_excel(writer, sheet_name=f'{promotor[:-8]}', index=True)
                
    def exportavel_so_cidades(self):
        agora = datetime.now()
        export = agora.strftime('%Y-%m-%d_%H-%M')
        pedidos = self._filtrar_pedidos()
        with pd.ExcelWriter(f"export/SO_cidades_{export}.xlsx", engine='xlsxwriter') as writer:
            gp = pedidos.groupby('cidade')
            df = pd.DataFrame()
            df['Pedidos'] = gp['id_pedido'].count()
            df['Total Deixado'] = gp['valor_total_pedido'].sum()
            df['Total Vendido'] = gp['valor_venda_pedido'].sum()
            df.to_excel(writer, sheet_name=f'cidades', index=True)
            
    def exportavel_desempenho_promotor_por_mes(self):
        agora = datetime.now()
        export = agora.strftime('%Y-%m-%d_%H-%M')
        desempenho = self._filtrar_desempenho()
        with pd.ExcelWriter(f"export/desempenho_promotor_{export}.xlsx", engine='xlsxwriter') as writer:
            for promotor in desempenho['descricao_vendedor'].unique():
                df = pd.DataFrame()
                desempenhoPromotor = desempenho[desempenho['descricao_vendedor'] == promotor]
                gp = desempenhoPromotor.groupby('ano_mes_cobranca')
                df['Pedidos Cobrados'] = gp['pedidos_fechados'].sum()
                df['Clientes novos'] = gp['clientes_novos'].sum()
                df['Deixado'] = gp['vl_total_pedidos_liquido'].sum()
                df['Cobrança'] = gp['valor_recebido'].sum()
                df['Pepinos'] = gp['total_novos_pepinos'].sum()
                df['Rec Pepino'] = gp['valor_pepino_recebidos'].sum()
                df.to_excel(writer, sheet_name=f'{promotor[:-8]}', index=True)
                
    def exportavel_desempenho_promotor_geral(self):
        agora = datetime.now()
        export = agora.strftime('%Y-%m-%d_%H-%M')
        desempenho = self._filtrar_desempenho()
        with pd.ExcelWriter(f"export/desempenho_promotores_{export}.xlsx", engine='xlsxwriter') as writer:
            df = pd.DataFrame()
            desempenhoPromotor = desempenho
            gp = desempenhoPromotor.groupby('descricao_vendedor')
            df['Pedidos Cobrados'] = gp['pedidos_fechados'].sum()
            df['Clientes novos'] = gp['clientes_novos'].sum()
            df['Deixado'] = gp['vl_total_pedidos_liquido'].sum()
            df['Cobrança'] = gp['valor_recebido'].sum()
            df['Pepinos'] = gp['total_novos_pepinos'].sum()
            df['Rec Pepino'] = gp['valor_pepino_recebidos'].sum()
            df.to_excel(writer, sheet_name=f'tabela', index=True)
                

In [ ]:
class internos(app):
    def __init__(self, main) -> None:
        super().__init__(main)
    
    def analise_v_m_v_a(self):
        sacoleirasAtivas = self._get_listaVendedorasAtivas()
        todasMedianas = pd.DataFrame()
        for sacoleira in sacoleirasAtivas:
            df = pd.DataFrame()
            histSacoleira = self.main.artigosPedido[self.main.artigosPedido['id_sacoleira'] == sacoleira]
            gpHist = histSacoleira.groupby(['ref', 'artigo'])
            df['Quantidade Vendida'] = gpHist['quantidade_vendida'].median()
            df['Quantidade Entregue'] = gpHist['quantidade_entregue'].median()
            todasMedianas = pd.concat([todasMedianas, df])
        df = todasMedianas.groupby(['ref', 'artigo']).sum()
        self._atualizar_exibicao({'nome': 'Relatorio', 'objeto': df})
        

In [ ]:
class vendedoras(app):
    def __init__(self, main) -> None:
        super().__init__(main)
    
    def _build_layout(self):
        self.selecao_vendedora = widgets.Dropdown(options=[],
                                description='Vendedora:',
                                disabled=False)
        self.selecao_vendedora.observe(self._vendedora_unica_selecionada, names='value') # type: ignore
        self.selecao_data_cobranca = widgets.Dropdown(options=[],
                                description='Data de Cobrança:')
        self.selecao_data_cobranca.observe(self._data_selecionada, names='value') # type: ignore
        #TODO FILTRO
        self.selecao_status = widgets.SelectMultiple(
            options=['FECHADO', 'DIGITADO', 'PEPINO'],
            value=['FECHADO', 'DIGITADO', 'PEPINO'],
            description='Status',
            disabled=False
        )
        self.box_vendedoras_todas_cabecalhos = widgets.HBox([self.selecao_data_cobranca, self.selecao_status])
        self.output_dataframe_vendedoras_todas_historico = widgets.Output()
        self.box_tab_vendedoras_todas = widgets.VBox([self.box_vendedoras_todas_cabecalhos, self.output_dataframe_vendedoras_todas_historico])
        self.dataframe_historico_vendedora = 0
        tab_vendedoras_contents = ['Todas']
        children = [self.box_tab_vendedoras_todas]
        self.tabs_vendedoras = widgets.Tab(children=children)
        for i, title in enumerate(tab_vendedoras_contents):
            self.tabs_vendedoras.set_title(i, title)
        self.layout = widgets.VBox(children=[self.selecao_vendedora, self.tabs_vendedoras])
        return self.layout
        
    def analise_ativas(self):
        pedidosDigitados = self._filtrar_pedidos('DIGITADO')
        listaAtivas = pedidosDigitados['id_sacoleira'].unique()
        pedidosTodos = self._filtrar_pedidos()
        pedidosTodos = pedidosTodos.query('id_sacoleira in @listaAtivas')
        self.pedidosTodos = pedidosTodos.merge(self.main.sacoleiras[['id_sacoleira', 'nome']], on='id_sacoleira')
        datasCobranca = pedidosTodos['ano_mes_cobranca'].unique()
        datasCobranca = sorted(datasCobranca, reverse=True)
        self.selecao_data_cobranca.options = datasCobranca
    
    def analise_cobradas(self):
        pedidosTodos = self._filtrar_pedidos()
        self.pedidosTodos = pedidosTodos.merge(self.main.sacoleiras[['id_sacoleira', 'nome']], on='id_sacoleira')
        datasCobranca = pedidosTodos['ano_mes_cobranca'].unique()
        datasCobranca = sorted(datasCobranca, reverse=True)
        self.selecao_data_cobranca.options = datasCobranca

    def _data_selecionada(self, value):
        self.dataDf = value['new']
        self.dataDf = self.dataDf.strftime("%m/%Y") # type: ignore
        # Convertendo as strings para o tipo datetime
        self.dataDf = datetime.strptime(self.dataDf, "%m/%Y").date()
        self._criar_dataframe_central()

    def _vendedora_unica_selecionada(self, value):
        value = value['new']
        self._criar_tab_vendedora(value)

    def _fechar_tab_vendedora(self, b):
        indice_tab_ativa = self.tabs_vendedoras.selected_index
        if indice_tab_ativa is not None and indice_tab_ativa >= 0:
            self.tabs_vendedoras.children = [
            child for i, child in enumerate(self.tabs_vendedoras.children) if i != indice_tab_ativa # type: ignore
        ]

    def _criar_tab_vendedora(self, sacoleira):
        button_fechar_aba = widgets.Button(description= 'fechar aba')
        button_fechar_aba.on_click(self._fechar_tab_vendedora)
        output_vendedora = widgets.Output()
        nova_aba = widgets.VBox(children=[button_fechar_aba, output_vendedora])
        self.tabs_vendedoras.children = list(self.tabs_vendedoras.children) + [nova_aba] # type: ignore
        self.tabs_vendedoras.set_title(len(self.tabs_vendedoras.children) - 1, f'{self.selecao_vendedora.label}')
        self._criar_relatorios_vendedora(sacoleira, output_vendedora)

    def _criar_relatorios_vendedora(self, sacoleira, output):
        def criarHistorico(sacoleira):
            dataframe_historico_vendedora = self.main.pedidos.query('id_sacoleira == @sacoleira')
            dataframe_historico_vendedora = dataframe_historico_vendedora[['ano_mes_cobranca', 'id_pedido', 'valor_total_pedido', 'valor_venda_pedido']].sort_values('id_pedido', ascending= True)
            dataframe_historico_vendedora['% Vendida'] = dataframe_historico_vendedora['valor_venda_pedido']/dataframe_historico_vendedora['valor_total_pedido']
            dataframe_historico_vendedora['id_pedido_anterior'] = dataframe_historico_vendedora['id_pedido'].shift(1)
            dataframe_historico_vendedora['Similaridade'] = 0.0
            for index, row in dataframe_historico_vendedora.iterrows():
                similaridade = _do_calcular_similaridade_pedidos(row)
                dataframe_historico_vendedora.at[index, 'Similaridade'] = similaridade
            dataframe_historico_vendedora = dataframe_historico_vendedora[['ano_mes_cobranca', 'id_pedido', 'valor_total_pedido', 'valor_venda_pedido','% Vendida', 'Similaridade' ]]
            dataframe_historico_vendedora = dataframe_historico_vendedora.rename(columns={
                'ano_mes_cobranca': 'Ano Mês Cobrança',
                'id_pedido': 'Pedido',
                'valor_total_pedido': 'Total Deixado',
                'valor_venda_pedido': 'Valor vendido',
            })
            style = {
                'Pedido': '{:.0f}',
                'Total Deixado': 'R$ {:,.2f}',
                'Valor vendido': 'R$ {:,.2f}',
                'Similaridade': '{:.2%}',
                '% Vendida': '{:.2%}'
            }
            return {'nome':'Historico de Vendas', 'objeto': dataframe_historico_vendedora, 'style': style}
        
        def _do_calcular_similaridade_pedidos(self, row):
            id_pedido1 = row['id_pedido']
            id_pedido2 = row['id_pedido_anterior']  # coluna que contém o id_pedido da linha anterior
            filtro = self.main.artigosPedido[self.main.artigosPedido['id_pedido'].isin([id_pedido1, id_pedido2])]
            referencias_pedido1 = filtro[filtro['id_pedido'] == id_pedido1]['ref'].tolist()
            referencias_pedido2 = filtro[filtro['id_pedido'] == id_pedido2]['ref'].tolist()
            # Calculando a similaridade de Jaccard entre as referências
            set_referencias1 = set(referencias_pedido1)
            set_referencias2 = set(referencias_pedido2)
            intersecao = set_referencias1.intersection(set_referencias2)
            uniao = set_referencias1.union(set_referencias2)
            similaridade = len(intersecao) / len(uniao)
            return similaridade       
        
        def _criarHistoricoVendas(sacoleira):
            itensFechados = self.main.artigosPedido.query(f'id_sacoleira == @sacoleira and status != "DIGITADO"')
            itensGroup = itensFechados.groupby(['ref', 'artigo'])
            df = pd.DataFrame()
            
            df['Quantidade Entregue'] = itensGroup['quantidade_entregue'].sum()
            df['Quantidade Vendida'] = itensGroup['quantidade_vendida'].sum()
            df['% Vendida'] = df['Quantidade Vendida']/df['Quantidade Entregue']
            df['Venda Mediana'] = itensGroup['quantidade_vendida'].mean()
            if self.main.artigosPedido.query(f'id_sacoleira == @sacoleira and status == "DIGITADO"').shape[0] > 0:
                itensDigitados = self.main.artigosPedido.query(f'id_sacoleira == @sacoleira and status == "DIGITADO"').groupby(['ref', 'artigo'])
                dfDigitado = pd.DataFrame()
                dfDigitado['Total Atual'] = itensDigitados['quantidade_entregue'].sum()
                df = df.merge(dfDigitado,  left_index=True, right_index=True, how='left')
            
            df.fillna(0, inplace=True)
            
            return {'nome':'Historico de Artigos', 'objeto': df}
        
        sacoleira = int(sacoleira)
        relatorioHistorico = criarHistorico(sacoleira)
        todosArtigos = _criarHistoricoVendas(sacoleira)
        
        self._atualizar_exibicao([relatorioHistorico,todosArtigos], output=output)

        
    def _criar_dataframe_central(self):
        def mainDf():
            dataframe_vendedoras_todas_historico = self.pedidosTodos
            dataframe_vendedoras_todas_historico = dataframe_vendedoras_todas_historico.query('ano_mes_cobranca == @self.dataDf')
            dataframe_vendedoras_todas_historico_copy = dataframe_vendedoras_todas_historico.copy()
            dataframe_vendedoras_todas_historico_copy['% Vendida'] = dataframe_vendedoras_todas_historico_copy['valor_venda_pedido'] / dataframe_vendedoras_todas_historico_copy['valor_total_pedido']
            dataframe_vendedoras_todas_historico_copy['Relevancia Cobrança'] = dataframe_vendedoras_todas_historico_copy['valor_venda_pedido'] / dataframe_vendedoras_todas_historico_copy['valor_venda_pedido'].sum()
            dataframe_vendedoras_todas_historico_copy['Relevancia Consignado'] = dataframe_vendedoras_todas_historico_copy['valor_total_pedido'] / dataframe_vendedoras_todas_historico_copy['valor_total_pedido'].sum()
            dataframe_vendedoras_todas_historico_copy['Ativa'] = dataframe_vendedoras_todas_historico_copy.apply(lambda row: 'sim' if any((self.pedidosTodos['id_sacoleira'] == row['id_sacoleira']) & (self.pedidosTodos['status'] == 'DIGITADO')) else 'não', axis=1)
            dataframe_vendedoras_todas_historico = dataframe_vendedoras_todas_historico_copy
            df_dadosExtra = self.main.pedidos.groupby('id_sacoleira').agg({'valor_venda_pedido': 'median', 'valor_total_pedido': 'median', 'id_pedido': 'count'}).reset_index()
            df_dadosExtra = df_dadosExtra.rename(columns= {'valor_venda_pedido': 'Venda Mediana', 'valor_total_pedido': 'Sacola Mediana', 'id_pedido': 'Total Pedidos'})
            dataframe_vendedoras_todas_historico = dataframe_vendedoras_todas_historico.merge(df_dadosExtra, on = 'id_sacoleira')
            dataframe_vendedoras_todas_historico = dataframe_vendedoras_todas_historico[['id_sacoleira','nome', 'Ativa', 'Total Pedidos', 'status', 'valor_total_pedido', 'Relevancia Consignado', 'valor_venda_pedido','Relevancia Cobrança','% Vendida', 'Venda Mediana', 'descricao_praca']]
            
            dataframe_vendedoras_todas_historico = dataframe_vendedoras_todas_historico.rename(columns={
                'id_sacoleira': 'Id',
                'nome': 'Nome',
                'status': 'Status',
                'valor_total_pedido': 'Valor Pedido',
                'valor_venda_pedido': 'Valor Vendido',
                'descricao_praca': 'Praca'
            })
            style = {
                'Valor Pedido':'R$ {:,.2f}' ,
                'Valor Vendido':'R$ {:,.2f}',
                'Venda Mediana':'R$ {:,.2f}',
                'Sacola Mediana':'R$ {:,.2f}',
                '% Vendida':'{:.2%}',
                'Relevancia Cobrança':'{:.2%}',
                'Relevancia Consignado':'{:.2%}',
                'Total Pedidos': '{:.0f}'
            }
            return {'nome': 'Vendedoras', 'objeto': dataframe_vendedoras_todas_historico, 'style': style}
        
        def _dfResumo(mainDf):
            datadf = pd.DataFrame()
            mainGb = mainDf.groupby('Praca')
            datadf['Vendedoras'] = mainGb['Total Pedidos'].count()
            datadf['Expectativa de Venda'] = mainGb['Venda Mediana'].sum()
            datadf['Total Vendido'] = mainGb['Valor Vendido'].sum()
            datadf['Resultado X Previsao'] = datadf['Total Vendido']/datadf['Expectativa de Venda']
            datadf['Venda Media'] = datadf['Total Vendido']/datadf['Vendedoras']
            datadf['Sacola Media'] = mainGb['Valor Pedido'].mean()
            datadf['% Vendida Media'] = datadf['Venda Media']/datadf['Sacola Media']
            style = {
                'Vendedoras':'{:,.2f}' ,
                'Expectativa de Venda':'R$ {:,.2f}',
                'Total Vendido':'R$ {:,.2f}',
                'Resultado X Previsao':'{:.2%}',
                'Venda Media':'R$ {:.2f}',
                'Sacola Media':'R$ {:.2f}',
                '% Vendida Media': '{:.0f}'}
            return {'nome': 'Resultado', 'objeto': datadf, 'style': style}
        
        def _graf_deixado_vendido(mainDf):
            fig, ax = plt.subplots(figsize=(10, 6))
            scatter_plot = sns.scatterplot(
                x='Valor Vendido',
                y='Valor Pedido',
                hue='Total Pedidos',  # Atribuir cores com base na coluna "Total Pedidos"
                palette='viridis',  # Escolha da paleta de cores (pode ser ajustada)
                data=mainDf,
                ax=ax
            )
            ax.set_title('Gráfico de Dispersão - Valor Pedido vs Valor Vendido')
            ax.set_xlabel('Valor Vendido')
            ax.set_ylabel('Valor Pedido')
            scatter_plot.legend(title='Total Pedidos')
            plt.close(fig)
            return fig
        
        def _graf_deixado_vendido_mediano(mainDf):
            fig, ax = plt.subplots(figsize=(10, 6))
            scatter_plot = sns.scatterplot(
                x='Venda Mediana',
                y='Sacola Mediana',
                hue='Total Pedidos',  # Atribuir cores com base na coluna "Total Pedidos"
                palette='viridis',  # Escolha da paleta de cores (pode ser ajustada)
                data=mainDf,
                ax=ax
            )
            ax.set_title('Gráfico de Dispersão - Venda Mediana vs Valor Pedido')
            ax.set_xlabel('Venda Mediana')
            ax.set_ylabel('Valor Pedido')
            scatter_plot.legend(title='Total Pedidos')
            plt.close(fig)
            return fig
        
        
        def _graf_pedidos_vendido(mainDf):
            fig, ax = plt.subplots(figsize=(10, 6))
            scatter_plot = sns.scatterplot(
                y='Valor Vendido',
                x='Total Pedidos',
                hue='Valor Pedido',  # Atribuir cores com base na coluna "Total Pedidos"
                palette='viridis',  # Escolha da paleta de cores (pode ser ajustada)
                data=mainDf,
                ax=ax
            )
            ax.set_title('Gráfico de Dispersão - Valor Vendido vs Total Pedidos')
            ax.set_ylabel('Valor Vendido')
            ax.set_xlabel('Total Pedidos')
            scatter_plot.legend(title='Valor Pedido')
            plt.close(fig)
            return fig
        
        def _graf_pedidos_deixado(mainDf):
            fig, ax = plt.subplots(figsize=(10, 6))
            scatter_plot = sns.scatterplot(
                y='Valor Pedido',
                x='Total Pedidos',
                hue='Valor Vendido',  # Atribuir cores com base na coluna "Total Pedidos"
                palette='viridis',  # Escolha da paleta de cores (pode ser ajustada)
                data=mainDf,
                ax=ax
            )
            ax.set_title('Gráfico de Dispersão - Valor Pedido vs Total Pedidos')
            ax.set_ylabel('Valor Pedido')
            ax.set_xlabel('Total Pedidos')
            scatter_plot.legend(title='Valor Vendido')
            plt.close(fig)
            return fig
        
        dataframePrincipal = mainDf()
        dataframeResumo = _dfResumo(dataframePrincipal['objeto'])
        graficos = {'nome': 'Graf. Pedidos x Vendido', 'objeto': [_graf_deixado_vendido(dataframePrincipal['objeto']), _graf_pedidos_vendido(dataframePrincipal['objeto'])]}
        self._atualizar_exibicao([dataframePrincipal, dataframeResumo, graficos], output= self.output_dataframe_vendedoras_todas_historico)
        self._carregar_lista_vendedoras(dataframePrincipal['objeto'])
            
    def _carregar_lista_vendedoras(self, df):
        listaVendedoras = []
        df.sort_values(by= 'Id', ascending = True, inplace = True)
        for _, linha in df.iterrows():
            busca = linha['Id']
            sacoleira = df.query('Id == @busca')
            tupla = (f"{sacoleira['Id'].to_string(index=False)} - {sacoleira['Nome'].to_string(index=False)}", sacoleira['Id'].to_string(index=False))
            listaVendedoras.append(tupla)
        self.selecao_vendedora.options = listaVendedoras        

In [ ]:
class artigos(app):
    def __init__(self, main) -> None:
        super().__init__(main)
        
    
    def resultado_de_artigos_periodo(self):
        df = pd.DataFrame()
        artigos_d = self._filtrar_artigosNoPedido(status='DIGITADO').groupby(['descricao_grupo','descricao_subgrupo', 'ref', 'artigo'])
        artigos_f = self._filtrar_artigosNoPedido().groupby(['descricao_grupo', 'descricao_subgrupo', 'ref', 'artigo'])
        df['Entregue'] = artigos_f['quantidade_entregue'].sum()
        df['Vendido'] = artigos_f['quantidade_vendida'].sum()
        df['Faturado'] = artigos_f['valor_vendido'].sum()
        df['Na Rua'] = artigos_d['quantidade_entregue'].sum()
        
        df_graf = df.reset_index()
        trace = go.Treemap(
            labels=df_graf['artigo'],
            parents=df_graf['descricao_subgrupo'],
            values=df_graf['Vendido']
        )
        fig = go.FigureWidget(trace)
        
        graf_vendido = px.treemap(df_graf, values='Vendido', path=['descricao_grupo', 'descricao_subgrupo', 'artigo'])
        graf_faturado = px.treemap(df_graf, values='Faturado', path=['descricao_grupo', 'descricao_subgrupo', 'artigo'])
        
        
        
        self._atualizar_exibicao([{'nome': 'Tabela', 'objeto': df}, {'nome': 'Grafico Quantidade', 'objeto': fig}])
        
    def resultado_de_grupos_periodo(self):
        df = pd.DataFrame()
        artigos_d = self._filtrar_artigosNoPedido(status='DIGITADO').groupby(['descricao_grupo'])
        artigos_f = self._filtrar_artigosNoPedido().groupby(['descricao_grupo'])
        df['Entregue'] = artigos_f['quantidade_entregue'].sum()
        df['Vendido'] = artigos_f['quantidade_vendida'].sum()
        df['Na Rua'] = artigos_d['quantidade_entregue'].sum()
        

        self._atualizar_exibicao({'nome': 'Tabela', 'objeto': df})

In [ ]:
class regionais(app):
    def __init__(self, main) -> None:
        super().__init__(main)
        
    def _gerar_heatmap(self, df,linha_valor, posicao = 'centro_marcacao' ):
        df_sem_nan = df.dropna(subset=['latitude', 'longitude', linha_valor])
        if posicao == 'bauru':
            mapa = folium.Map(location=[-22.3154, -49.0615], zoom_auto=True)
        if posicao == 'centro_regional':
            mapa = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_auto=True)
        heat_data = [[row['latitude'], row['longitude'], row[linha_valor]] for index, row in df_sem_nan.iterrows()]
        HeatMap(heat_data).add_to(mapa)        
        return mapa
    
    def mapa_de_calor_vendas(self):
        pedidos = self._filtrar_pedidos(status='FECHADO')    
        mapa = self._gerar_heatmap(pedidos, 'valor_venda_pedido', posicao= 'bauru')
        cidades = pedidos.groupby('cidade')
        tabela_cidades = pd.DataFrame()
        tabela_cidades['Total Venda'] = cidades['valor_venda_pedido'].sum()
        tabela_cidades['Total Entregue'] = cidades['valor_total_pedido'].sum()
        tabela_cidades['Quantidade (Notas)'] = cidades['id_pedido'].count()
        tabela_cidades['Venda/Nota'] = tabela_cidades['Total Venda'] / tabela_cidades['Quantidade (Notas)']
        tabela_cidades['% Vendida'] = tabela_cidades['Total Venda'] / tabela_cidades['Total Entregue']
        tabela_cidades['latitude'] = cidades['latitude'].mean()
        tabela_cidades['longitude'] = cidades['longitude'].mean()

        coordinates = tabela_cidades[['latitude', 'longitude']]
        num_clusters = 7
        kmeans = KMeans(n_clusters=num_clusters, random_state=42)
        tabela_cidades['regiao'] = kmeans.fit_predict(coordinates)

        for index, row in tabela_cidades.iterrows():
            folium.Marker([row['latitude'], row['longitude']], popup=f"Região: {row['regiao']}").add_to(mapa)

        tabela_cidades = tabela_cidades.drop(['latitude', 'longitude'], axis=1)
        self._atualizar_exibicao([{'nome':"Mapa", 'objeto':mapa}, {'nome':"Tabela Cidades", 'objeto': tabela_cidades}])

    def mapa_de_calor_entregue(self):
        pedidos = self._filtrar_pedidos(status='DIGITADO')    
        mapa = self._gerar_heatmap(pedidos, 'valor_total_pedido', posicao= 'bauru')
        cidades = pedidos.groupby('cidade')
        tabela_cidades = pd.DataFrame()
        tabela_cidades['Total Entregue'] = cidades['valor_total_pedido'].sum()
        tabela_cidades['Quantidade (Notas)'] = cidades['id_pedido'].count()
        tabela_cidades['Entregue/Nota'] = tabela_cidades['Total Entregue'] / tabela_cidades['Quantidade (Notas)']
        tabela_cidades['latitude'] = cidades['latitude'].mean()
        tabela_cidades['longitude'] = cidades['longitude'].mean()

        coordinates = tabela_cidades[['latitude', 'longitude']]
        num_clusters = 7
        kmeans = KMeans(n_clusters=num_clusters, random_state=42)
        tabela_cidades['regiao'] = kmeans.fit_predict(coordinates)

        for index, row in tabela_cidades.iterrows():
            folium.Marker([row['latitude'], row['longitude']], popup=f"Região: {row['regiao']}").add_to(mapa)

        tabela_cidades = tabela_cidades.drop(['latitude', 'longitude'], axis=1)
        self._atualizar_exibicao([{'nome':"Mapa", 'objeto':mapa}, {'nome':"Tabela Cidades", 'objeto': tabela_cidades}])
    
    def mapa_de_calor_vendas_com_praca(self):
        pedidos = self._filtrar_pedidos(status='FECHADO')    
        mapa = self._gerar_heatmap(pedidos, 'valor_venda_pedido', posicao= 'bauru')
        cidades = pedidos.groupby(['cidade','descricao_praca'])
        tabela_cidades = pd.DataFrame()
        tabela_cidades['Total Venda'] = cidades['valor_venda_pedido'].sum()
        tabela_cidades['Total Entregue'] = cidades['valor_total_pedido'].sum()
        tabela_cidades['Quantidade (Notas)'] = cidades['id_pedido'].count()
        tabela_cidades['Venda/Nota'] = tabela_cidades['Total Venda'] / tabela_cidades['Quantidade (Notas)']
        tabela_cidades['% Vendida'] = tabela_cidades['Total Venda'] / tabela_cidades['Total Entregue']
        tabela_cidades['latitude'] = cidades['latitude'].mean()
        tabela_cidades['longitude'] = cidades['longitude'].mean()

        coordinates = tabela_cidades[['latitude', 'longitude']]
        num_clusters = 7
        kmeans = KMeans(n_clusters=num_clusters, random_state=42)
        tabela_cidades['regiao'] = kmeans.fit_predict(coordinates)

        for index, row in tabela_cidades.iterrows():
            folium.Marker([row['latitude'], row['longitude']], popup=f"Região: {row['regiao']}").add_to(mapa)

        tabela_cidades = tabela_cidades.drop(['latitude', 'longitude'], axis=1)
        self._atualizar_exibicao([{'nome':"Mapa", 'objeto':mapa}, {'nome':"Tabela Cidades", 'objeto': tabela_cidades}])

    def mapa_de_calor_entregue_com_praca(self):
        pedidos = self._filtrar_pedidos(status='DIGITADO')  
        mapa = self._gerar_heatmap(pedidos, 'valor_total_pedido', posicao= 'bauru')
        cidades = pedidos.groupby(['cidade','descricao_praca'])
        tabela_cidades = pd.DataFrame()
        tabela_cidades['Total Entregue'] = cidades['valor_total_pedido'].sum()
        tabela_cidades['Quantidade (Notas)'] = cidades['id_pedido'].count()
        tabela_cidades['latitude'] = cidades['latitude'].mean()
        tabela_cidades['longitude'] = cidades['longitude'].mean()
        coordinates = tabela_cidades[['latitude', 'longitude']]
        num_clusters = 7
        kmeans = KMeans(n_clusters=num_clusters, random_state=42)
        tabela_cidades['regiao'] = kmeans.fit_predict(coordinates)

        for index, row in tabela_cidades.iterrows():
            folium.Marker([row['latitude'], row['longitude']], popup=f"Região: {row['regiao']}").add_to(mapa)
        
        """
        for index, row in tabela_cidades.iterrows():
            cidade = index[0]
            pracas = tabela_cidades[tabela_cidades.index.get_level_values('cidade') == cidade]['descricao_praca'].unique()
            folium.Marker([row['latitude'], row['longitude']], popup=f"Região: {row['reg
            iao']}\nPraça(s){pracas}").add_to(mapa)   
        """
        tabela_cidades = tabela_cidades.drop(['latitude', 'longitude'], axis=1)
        self._atualizar_exibicao([{'nome':"Mapa", 'objeto':mapa}, {'nome':"Tabela Cidades", 'objeto': tabela_cidades}])

In [ ]:
class inteligenciaArtificial(app):
    def __init__(self, main) -> None:
        super().__init__(main)
    
    

In [ ]:
class MainApp:
    def __init__(self):
        self.artigos = artigos(self)
        self.vendedoras = vendedoras(self)
        self.internos = internos(self)
        self.relatorios = relatorio(self)
        self.regionais = regionais(self)
        self.apps = [('Artigos', self.artigos),('Vendedoras', self.vendedoras),('Analises Regionais', self.regionais), ('Processos Internos', self.internos), ('Relatorios', self.relatorios)]
        self.load_data()
        self.create_layout()         
        self.tabs_data = []
        
    def load_data(self):
        self.vendedores = pd.read_csv('data/vendedores.csv', sep=';')
        self.pracas = pd.read_csv('data/pracas.csv', sep=';')
        self.desempenhoPraca = pd.read_csv('data/desempenhoPraca.csv', sep=';')
        self.pedidos = pd.read_csv('data/pedidos.csv', sep=';')
        self.sacoleiras = pd.read_csv('data/sacoleiras.csv', sep=';')
        self.artigosPedido = pd.read_csv('data/artigosPedido.csv', sep=';')
        self.gruposArtigos = pd.read_csv('data/grupos.csv', sep=';')
        self.artigos = pd.read_csv('data/artigos.csv', sep=';')
        self.infoCidades = pd.read_csv('data/cidades.csv', sep=';')
        
        self.artigosPedido['ano_mes_cobranca'] = pd.to_datetime(self.artigosPedido['ano_mes_cobranca'], format='%m/%Y').dt.date
        self.desempenhoPraca['ano_mes_cobranca'] = pd.to_datetime(self.desempenhoPraca['ano_mes_cobranca'], format='%Y-%m').dt.date
        self.pedidos['ano_mes_cobranca'] = pd.to_datetime(self.pedidos['ano_mes_cobranca'], format='%m/%Y').dt.date

    def create_layout(self):
        # Promotor
        self.check_soPromotorEPracaAtivo = widgets.Checkbox(description="Só Ativas?", value=True)
        listaPromotores = self.get_lista_promotores()
        self.select_promotores = widgets.SelectMultiple(description='Promotor(es)', options=listaPromotores, value=[promotor[1] for promotor in listaPromotores])
        self.check_todosPromotores = widgets.Checkbox(description="Todos", value=False)
        self.box_promotores = widgets.HBox(children=[self.select_promotores, self.check_todosPromotores])
        
        # Praça
        listaPracas = self.get_lista_pracas()
        self.text_id_pracas = widgets.Text(description='Praça(s) [id]')
        self.select_pracas = widgets.SelectMultiple(description='Praça(s)', options=listaPracas, value=[praca[1] for praca in listaPracas])
        self.check_todasPracas = widgets.Checkbox(description="Todas", value=False)
        self.box_pracas = widgets.HBox(children=[self.text_id_pracas, self.select_pracas, self.check_todasPracas])
        self.box_promotorPraca = widgets.HBox(children=[self.box_promotores, self.box_pracas, self.check_soPromotorEPracaAtivo])
        
        # Periodo
        self.dateP_dataInicial = widgets.DatePicker(description='Data Inicial Cobrança', value=dt.date.today() - dt.timedelta(days=2 * 30))
        self.dateP_dataFinal = widgets.DatePicker(description='Data Final Cobrança', value=dt.date.today() + dt.timedelta(days=2 * 30))
        self.box_periodo = widgets.HBox(children=[self.dateP_dataInicial, self.dateP_dataFinal])
        
        # Grupos e Artigos
        listaGrupos = self.get_lista_grupos()
        self.select_gruposArtigos = widgets.SelectMultiple(description='Grupos', options=listaGrupos, value=[grupo[1] for grupo in listaGrupos])
        self.check_todosGrupos = widgets.Checkbox(description='Todos', value=False)
        listaArtigos = self.get_lista_artigos()
        self.text_ref_artigos = widgets.Text(description='Artigos(s) [ref]')
        self.select_artigos = widgets.SelectMultiple(description='Artigos', options=listaArtigos, value=[artigo[1] for artigo in listaArtigos])
        self.check_todosArtigos = widgets.Checkbox(description='Todos', value=False)
        self.box_gruposArtigos = widgets.HBox(children=[self.select_gruposArtigos, self.check_todosGrupos,self.text_ref_artigos, self.select_artigos, self.check_todosArtigos])

        # Relatórios
        self.select_app = widgets.Dropdown(description='App:', options=self.apps)
        self.select_funcao = widgets.Dropdown(description='Função:', options=self.get_lista_funcoes())
        self.togle_modoExibicao = widgets.ToggleButtons(
            options=['Mesma Tab', 'Nova Tab'],
            description='Relatório na:',
            value='Mesma Tab',
            tooltips=['O relatório substituirá a tab atual', 'O relatório entrará em uma nova tab'],
        )
        self.button_rodar = widgets.Button(description= 'Rodar')
        self.button_rodar.on_click(self._rodar)
        self.button_fechar = widgets.Button(description= 'Fechar Tab')
        self.button_fechar.on_click(self.fecharTabAtiva)
        self.box_apps = widgets.HBox(children=[self.select_app, self.select_funcao, self.togle_modoExibicao, self.button_rodar, self.button_fechar])

        self.filtros = [self.box_promotorPraca, self.box_periodo, self.box_gruposArtigos]
        self.box_filtros = widgets.VBox(children=self.filtros)
        self.tab_apps = widgets.Tab()
        self.box_fixos = [self.box_filtros, self.box_apps, self.tab_apps]
        self.box_main = widgets.VBox(children=self.box_fixos)
        # Add observers to all widgets
        self.add_observer_to_widgets(self.box_filtros)
        # Display the nested boxes
        display(self.box_main)
    
    def get_lista_funcoes(self):
        chamado = getattr(self.select_app.value, '_retornar_funcoes')
        lista = chamado()
        return lista
    
    def get_lista_promotores(self):
        listaPromotores = []
        promotores = self.vendedores.sort_values(by='descricao_vendedor', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            promotores = promotores.query('status_vendedor == "ATIVO"')
        for _, linha in promotores.iterrows():
            tupla = (linha['descricao_vendedor'], linha['id_vendedor'])
            listaPromotores.append(tupla)
        return listaPromotores
    
    #TODO Juntar função get_lista_pracas e get_lista_id_pracas
    def get_lista_pracas(self):
        listaPracas = []
        promotores = self.select_promotores.value
        pracas = self.pracas.query('id_vendedor.isin(@promotores)')
        pracas = pracas.sort_values(by='descricao_praca', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            pracas = pracas.query('status_praca == "ATIVO"')
        for _, linha in pracas.iterrows():
            tupla = (linha['descricao_praca'], linha['id_praca'])
            listaPracas.append(tupla)
        return listaPracas
    
    def _so_promotor_praca_ativo(self, _):
        self.select_promotores.options = self.get_lista_promotores()
    
    def get_lista_id_pracas(self):
        listaPracas = []
        promotores = self.select_promotores.value
        pracas = self.pracas.query('id_vendedor.isin(@promotores)')
        pracas = pracas.sort_values(by='descricao_praca', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            pracas = pracas.query('status_praca == "ATIVO"')
        for _, linha in pracas.iterrows():
            tupla = (linha['id_praca'])
            listaPracas.append(tupla)
        return listaPracas

    def get_lista_grupos(self):
        listaGrupos = []
        grupos = self.gruposArtigos
        grupos = grupos.sort_values(by='descricao_grupo', ascending=True)

        for _, linha in grupos.iterrows():
            tupla = (linha['descricao_grupo'], linha['id_grupo'])
            listaGrupos.append(tupla)

        return listaGrupos
    
    #TODO Juntar função get_lista_artigos e get_lista_ref_artigos
    def get_lista_artigos(self):
        listaArtigos = []
        grupos = self.select_gruposArtigos.value
        artigos = self.artigos.query('id_grupo.isin(@grupos)')
        artigos = artigos.sort_values(by='artigo', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            artigos = artigos.query('status_artigo == "ATIVO"')
        for _, linha in artigos.iterrows():
            tupla = (linha['artigo'], linha['ref'])
            listaArtigos.append(tupla)
        return listaArtigos
    
    def get_lista_ref_artigos(self):
        listaArtigos = []
        grupos = self.select_gruposArtigos.value
        artigos = self.artigos.query('id_grupo.isin(@grupos)')
        artigos = artigos.sort_values(by='artigo', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            artigos = artigos.query('status_artigo == "ATIVO"')
        for _, linha in artigos.iterrows():
            tupla = (linha['ref'])
            listaArtigos.append(tupla)
        return listaArtigos

    def add_observer_to_widgets(self, widget):
        # Specific functions
        self.check_soPromotorEPracaAtivo.observe(self._so_promotor_praca_ativo, names='value') #type: ignore
        self.select_promotores.observe(self._promotor_selecionado, names='value') # type: ignore
        self.select_gruposArtigos.observe(self._grupos_selecionados, names='value') # type: ignore
        self.check_todosPromotores.observe(self._selecionar_todos_promotores, names='value') # type: ignore
        self.check_todasPracas.observe(self._selecionar_todas_pracas, names='value') # type: ignore
        self.check_todosGrupos.observe(self._selecionar_todos_grupos, names='value') # type: ignore
        self.check_todosArtigos.observe(self._selecionar_todos_artigos, names='value') # type: ignore
        self.select_app.observe(self._app_selecionado, names='value') # type: ignore
        self.text_id_pracas.observe(self.id_praca_escrito, names='value') # type: ignore
        self.text_ref_artigos.observe(self.ref_artigo_escrito, names='value') # type: ignore
        
        # General function
        if isinstance(widget, (widgets.Box, widgets.VBox, widgets.HBox, widgets.GridBox)):
            for child_widget in widget.children: # type: ignore
                self.add_observer_to_widgets(child_widget)
        elif isinstance(widget, widgets.Widget):
            widget.observe(self.normal_object_updated, names='value') # type: ignore

    def normal_object_updated(self, change):
        if change['new'] == ():
            pass
        else:
            self._update_termos()

    def id_praca_escrito(self, change):
        valor = change['new']
        if valor == '':
            return
        padrao = re.compile(r'^[0-9,]*$')
        lista_pracas = self.get_lista_id_pracas()
        if padrao.match(valor):
            if valor[-1] != ',':
                pracasSelecionadasLista = [int(numero) for numero in valor.split(',')]
                pracasSelecionadas = []
                for praca in pracasSelecionadasLista:
                    if praca in lista_pracas:
                        pracasSelecionadas.append(praca)
                self.select_pracas.value = pracasSelecionadas
        else:
            self.text_id_pracas.value = change['old']
        
    def ref_artigo_escrito(self, change):
        valor = change['new']
        if valor == '':
            return
        padrao = re.compile(r'^[0-9,]*$')
        lista_artigos = self.get_lista_ref_artigos()
        if padrao.match(valor):
            if valor[-1] != ',':
                artigosSelecionadosLista = [int(numero) for numero in valor.split(',')]
                artigosSelecionados = []
                for praca in artigosSelecionadosLista:
                    if praca in lista_artigos:
                        artigosSelecionados.append(praca)
                self.select_artigos.value = artigosSelecionados
        else:
            self.text_ref_artigos.value = change['old']
        
    def _update_termos(self):
        # Convertendo as strings das datas para o formato "mm/aaaa"
        data_inicial_str = self.dateP_dataInicial.value.strftime("%m/%Y") # type: ignore
        data_final_str = self.dateP_dataFinal.value.strftime("%m/%Y") # type: ignore
        # Convertendo as strings para o tipo datetime
        data_inicial = datetime.strptime(data_inicial_str, "%m/%Y").date()
        data_final = datetime.strptime(data_final_str, "%m/%Y").date()
        self.filtros = {
            'promotores': self.select_promotores.value,
            'pracas': self.select_pracas.value,
            'data inicial': data_inicial,
            'data final': data_final,
            'artigos': self.select_artigos.value
        }
    
    def _updateOptionsPracas(self):
        self.select_pracas.options = self.get_lista_pracas()
        self.select_pracas.value = [praca[1] for praca in self.get_lista_pracas()]

    def _updateOptionsArtigos(self):
        self.select_artigos.options = self.get_lista_artigos()
        self.select_artigos.value = [artigo[1] for artigo in self.get_lista_artigos()]

    def _updateOptionsFuncoes(self):
        self.select_funcao.options = self.get_lista_funcoes()

    def _promotor_selecionado(self, change):
        self._updateOptionsPracas()

    def _grupos_selecionados(self, change):
        self._updateOptionsArtigos()

    def _app_selecionado(self, change):
        self._updateOptionsFuncoes()
        self._update_termos()

    def _rodar(self, _):
        def rodarAppNaTabAtiva(self):
            indice_tab_ativa = self.tab_apps.selected_index
            
            # Verifica se há alguma tab aberta
            if indice_tab_ativa is not None and indice_tab_ativa >= 0:
                tab_ativa = self.tab_apps.children[indice_tab_ativa] # type: ignore
                selected_app = self.select_app.value
                app_widgets = selected_app._build_layout() # type: ignore
                tab_ativa.children = [] + [app_widgets]
                funcao_selecionada = getattr(self.select_app.value, self.select_funcao.value) # type: ignore
                funcao_selecionada()
                self.tab_apps.set_title(len(self.tab_apps.children) - 1, f'{self.select_funcao.label}') # type: ignore
            else:
                rodarAppEmNovaTab(self)
                
        def rodarAppEmNovaTab(self):
            selected_app = self.select_app.value
            app_widgets = selected_app._build_layout() # type: ignore
            nova_aba = widgets.VBox([app_widgets])
            self.tab_apps.children = list(self.tab_apps.children) + [nova_aba] # type: ignore
            self.tab_apps.set_title(len(self.tab_apps.children) - 1, f'{self.select_funcao.label}')
            funcao_selecionada = getattr(self.select_app.value, self.select_funcao.value) # type: ignore
            funcao_selecionada()
                
        self._update_termos()
        if self.togle_modoExibicao.value == 'Mesma Tab':
            rodarAppNaTabAtiva(self)
        elif self.togle_modoExibicao.value == 'Nova Tab':
            rodarAppEmNovaTab(self)
    
    def fecharTabAtiva(self, _):
        indice_tab_ativa = self.tab_apps.selected_index
        if indice_tab_ativa is not None and indice_tab_ativa >= 0:
            self.tab_apps.children = [
            child for i, child in enumerate(self.tab_apps.children) if i != indice_tab_ativa # type: ignore
        ]
        
    def _selecionar_todos_promotores(self, change):
        if change['new']:
            self.select_promotores.value = [promotor[1] for promotor in self.get_lista_promotores()]
        else:
            self.select_promotores.value = []

    def _selecionar_todas_pracas(self, change):
        if change['new']:
            self.select_pracas.value = [praca[1] for praca in self.get_lista_pracas()]
        else:
            self.select_pracas.value = []

    def _selecionar_todos_grupos(self, change):
        if change['new']:
            self.select_gruposArtigos.value = [grupo[1] for grupo in self.get_lista_grupos()]
        else:
            self.select_gruposArtigos.value = []

    def _selecionar_todos_artigos(self, change):
        if change['new']:
            self.select_artigos.value = [artigo[1] for artigo in self.get_lista_artigos()]
        else:
            self.select_artigos.value = []


In [ ]:
oi = MainApp()

In [ ]:
pd.set_option('display.max_columns', None)
